In [ ]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

In [ ]:
# Don't run this cell when debugging from VS Code, only when running the notebook with Jupyter lab.
import debugpy
debugpy.listen(5678)
print("Waiting for debugger attach...")
debugpy.wait_for_client()
print("Debugger attached.")

In [ ]:
import urllib.request
import os.path

data_url = "https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad"
data_file = "pbmc3k.h5ad"

if not os.path.exists(data_file):
    print("Downloading PBMC 3K demo...")
    urllib.request.urlretrieve(data_url, data_file)
    print("Download complete.")

In [ ]:
import scanpy as sc

adata = sc.read(data_file)
adata

In [ ]:
from scsketch import ScSketch

sketch = ScSketch(
    adata=adata,
    metadata_cols=["louvain"],
    color_by_default="louvain",
    height=720,
    background_color="#111111",
)

sketch.show()